In [1]:
import matplotlib.pyplot as plt

In [3]:
from itertools import chain

import nltk
import sklearn
import scipy.stats
from sklearn.metrics import make_scorer
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RandomizedSearchCV

import sklearn_crfsuite
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics

A simple sentence NER example:

[**ORG** U.N. ] official [**PER** Ekeus ] heads for [**LOC** Baghdad ] 

We will concentrate on four types of named entities:
 * persons (**PER**), 
 * locations (**LOC**) 
 * organizations (**ORG**)
 * Others (**O**)

In [4]:
def _generate_examples(filepath):
        with open(filepath, encoding="utf-8") as f:
            sent = []
            for line in f:
                if line.startswith("-DOCSTART-") or line == "" or line == "\n":
                    if sent:
                        yield sent
                        sent = []
                else:
                    splits = line.split(" ")
                    token = splits[0]
                    pos_tag = splits[1]
                    ner_tag = splits[3].rstrip()
                    if 'MISC' in ner_tag:
                        ner_tag = 'O'
                    
                    sent.append((token, pos_tag, ner_tag))

In [6]:
import os
os.getcwd()

'/home/qlr/Programming/UCA_MSc-DS1/2_ML_algos/lecture_5/CRF_ML_ALGOS'

In [8]:
%%time
train_sents = list(_generate_examples(os.getcwd()+'/train.txt'))
test_sents = list(_generate_examples(os.getcwd()+'/test.txt'))

CPU times: user 242 ms, sys: 20 ms, total: 262 ms
Wall time: 261 ms


In [9]:
def word2features(sent, i):
    word = sent[i][0]
    postag = sent[i][1]

    features = {
        'bias': 1.0,
        'word.lower()': word.lower(),
        'postag': postag,
    }
    
    if i > 0:
        word1 = sent[i-1][0]
        postag1 = sent[i-1][1]
        features.update({
            '-1:word.lower()': word1.lower(),
            '-1:postag': postag1,
        })
    else:
        features['BOS'] = True
    return features

In [10]:
test_sents[2]

[('United', 'NNP', 'B-LOC'),
 ('Arab', 'NNP', 'I-LOC'),
 ('Emirates', 'NNPS', 'I-LOC'),
 ('1996-12-06', 'CD', 'O')]

In [11]:
word2features(test_sents[2],0)

{'bias': 1.0, 'word.lower()': 'united', 'postag': 'NNP', 'BOS': True}

In [12]:
def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
    return [label for token, postag, label in sent]

def sent2tokens(sent):
    return [token for token, postag, label in sent]

In [13]:
%%time
X_train = [sent2features(s) for s in train_sents]
y_train = [sent2labels(s) for s in train_sents]

X_test = [sent2features(s) for s in test_sents]
y_test = [sent2labels(s) for s in test_sents]

CPU times: user 255 ms, sys: 10.4 ms, total: 266 ms
Wall time: 265 ms


In [14]:
%%time
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    c1=0.1,
    c2=0.1,
    max_iterations=100,
    all_possible_transitions=True
)
crf.fit(X_train, y_train)

CPU times: user 11.1 s, sys: 49.1 ms, total: 11.1 s
Wall time: 11.2 s


/home/qlr/anaconda3/lib/python3.8/site-packages/sklearn/base.py:209: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  warnings.warn('From version 0.24, get_params will raise an '


CRF(algorithm='lbfgs', all_possible_transitions=True, c1=0.1, c2=0.1,
    keep_tempfiles=None, max_iterations=100)

In [15]:
labels = list(crf.classes_)

In [16]:
labels

['B-ORG', 'O', 'B-PER', 'I-PER', 'B-LOC', 'I-ORG', 'I-LOC']

In [17]:
labels.remove('O')

In [18]:
y_pred = crf.predict(X_test)
metrics.flat_f1_score(y_test, y_pred,
                      average='weighted', labels=labels)

0.7757476721426669

In [22]:
# group B and I results
sorted_labels = sorted(
    labels,
    key=lambda name: (name[1:], name[0])
)


In [23]:
print(metrics.flat_classification_report(
    y_test, y_pred, labels=sorted_labels, digits=3
))

/home/qlr/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:67: FutureWarning: Pass labels=['B-LOC', 'I-LOC', 'B-ORG', 'I-ORG', 'B-PER', 'I-PER'] as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "


              precision    recall  f1-score   support

       B-LOC      0.849     0.825     0.837      1667
       I-LOC      0.767     0.716     0.740       257
       B-ORG      0.735     0.637     0.682      1660
       I-ORG      0.616     0.721     0.664       834
       B-PER      0.837     0.764     0.799      1615
       I-PER      0.832     0.931     0.878      1156

   micro avg      0.785     0.769     0.777      7189
   macro avg      0.772     0.766     0.767      7189
weighted avg      0.787     0.769     0.776      7189



In [24]:
len(crf.transition_features_)

49

In [25]:
from collections import Counter

def print_transitions(trans_features):
    for (label_from, label_to), weight in trans_features:
        print("%-6s -> %-7s %0.6f" % (label_from, label_to, weight))

print("Top likely transitions:")
print_transitions(Counter(crf.transition_features_).most_common(20))

print("\nTop unlikely transitions:")
print_transitions(Counter(crf.transition_features_).most_common()[-20:])

Top likely transitions:
B-PER  -> I-PER   6.591492
B-ORG  -> I-ORG   6.306534
I-ORG  -> I-ORG   5.540077
B-LOC  -> I-LOC   4.839887
I-LOC  -> I-LOC   3.758774
I-PER  -> I-PER   3.394919
O      -> B-PER   1.960743
O      -> O       1.369676
B-ORG  -> O       0.950664
O      -> B-LOC   0.919982
B-LOC  -> O       0.612921
B-PER  -> O       0.557646
O      -> B-ORG   0.515605
I-PER  -> O       0.393510
I-ORG  -> O       0.328486
I-LOC  -> O       -0.305074
B-ORG  -> B-ORG   -0.984217
B-LOC  -> B-LOC   -0.990422
I-LOC  -> B-LOC   -1.291094
B-PER  -> B-LOC   -1.315197

Top unlikely transitions:
I-PER  -> I-ORG   -1.991960
I-PER  -> I-LOC   -2.040264
B-ORG  -> I-LOC   -2.046612
I-ORG  -> B-LOC   -2.113390
I-ORG  -> I-LOC   -2.206704
B-ORG  -> B-LOC   -2.584979
I-LOC  -> B-ORG   -2.658712
I-PER  -> B-LOC   -2.709911
B-LOC  -> B-PER   -2.738535
I-LOC  -> B-PER   -2.756531
I-LOC  -> I-ORG   -2.759703
B-PER  -> B-ORG   -2.798284
I-ORG  -> B-ORG   -3.030456
I-PER  -> B-ORG   -3.144592
I-PER  -> B-

In [26]:
len(crf.state_features_)

16044

In [27]:
def print_state_features(state_features):
    for (attr, label), weight in state_features:
        print("%0.6f %-8s %s" % (weight, label, attr))

print("Top positive:")
print_state_features(Counter(crf.state_features_).most_common(30))

print("\nTop negative:")
print_state_features(Counter(crf.state_features_).most_common()[-30:])

Top positive:
8.307293 I-LOC    word.lower():oval
8.088441 B-LOC    word.lower():m3
7.751193 B-ORG    word.lower():footscray
7.001409 B-ORG    word.lower():osce
6.964246 B-PER    word.lower():lebed
6.609227 B-LOC    word.lower():amsterdam
6.556081 B-LOC    word.lower():bonn
6.543649 B-LOC    word.lower():beijing
6.516252 B-LOC    word.lower():mideast
6.514129 B-ORG    word.lower():adelaide
6.341000 B-LOC    word.lower():balkans
6.296972 B-LOC    word.lower():med
6.293931 B-LOC    word.lower():stansted
6.251496 O        word.lower():to
6.164668 O        word.lower():division
6.109046 B-LOC    word.lower():vatican
6.065353 B-LOC    word.lower():johannesburg
6.051745 B-PER    word.lower():stenning
6.033868 B-LOC    word.lower():england
6.013143 B-PER    word.lower():clinton
5.989218 B-PER    word.lower():chang
5.986145 B-LOC    word.lower():pakistan
5.979900 B-LOC    word.lower():mt
5.896624 B-PER    word.lower():fogarty
5.840211 B-LOC    word.lower():moscow
5.801627 B-ORG    word.lower()